### Getting countries

In [4]:
import requests
import pandas as pd
import json


contest_url = "https://eurovisionapi.runasp.net/api/countries"

contests_response = requests.get(contest_url)

if contests_response.status_code == 200:
    contest = contests_response.json()
    print("Countries retrieved successfully:")

    with open("countries.json", "w", encoding="utf-8") as file:
        json.dump(contest, file, ensure_ascii=False, indent=4)
    
    print("Countries data saved to 'countries.json'")
else:
    print(f"Failed to retrieve data. Status code: {contests_response.status_code}")


Countries retrieved successfully:
Countries data saved to 'countries.json'


### Getting contests

In [5]:
import requests
import pandas as pd
import json


contest_url = "https://eurovisionapi.runasp.net/api/contests"

contests_response = requests.get(contest_url)

if contests_response.status_code == 200:
    contest = contests_response.json()
    print("Contests retrieved successfully:")

    with open("contests.json", "w", encoding="utf-8") as file:
        json.dump(contest, file, ensure_ascii=False, indent=4)
    
    print("Contest data saved to 'contests.json'")
else:
    print(f"Failed to retrieve data. Status code: {contests_response.status_code}")


Contests retrieved successfully:
Contest data saved to 'contests.json'


In [ ]:
import os

output_dir = "contests_details"
os.makedirs(output_dir, exist_ok=True)

for year in range(1956, 2025):
    url = f"https://eurovisionapi.runasp.net/api/contests/{year}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        file_path = os.path.join(output_dir, f"contest_{year}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Data for {year} saved to {file_path}")
    else:
        print(f"Failed to retrieve data for {year}. Status code: {response.status_code}")


Data for 1956 saved to contests_details\contest_1956.json
Data for 1957 saved to contests_details\contest_1957.json
Data for 1958 saved to contests_details\contest_1958.json
Data for 1959 saved to contests_details\contest_1959.json
Data for 1960 saved to contests_details\contest_1960.json
Data for 1961 saved to contests_details\contest_1961.json
Data for 1962 saved to contests_details\contest_1962.json
Data for 1963 saved to contests_details\contest_1963.json
Data for 1964 saved to contests_details\contest_1964.json
Data for 1965 saved to contests_details\contest_1965.json
Data for 1966 saved to contests_details\contest_1966.json
Data for 1967 saved to contests_details\contest_1967.json
Data for 1968 saved to contests_details\contest_1968.json
Data for 1969 saved to contests_details\contest_1969.json
Data for 1970 saved to contests_details\contest_1970.json
Data for 1971 saved to contests_details\contest_1971.json
Data for 1972 saved to contests_details\contest_1972.json
Data for 1973 

In [2]:

contests_dir = "contests_details"
contestants_dir = "contestants"

for year in range(1956, 2025):
    contest_file = os.path.join(contests_dir, f"contest_{year}.json")
    
    if not os.path.exists(contest_file):
        print(f"Contest file for {year} not found. Skipping.")
        continue

    with open(contest_file, "r", encoding="utf-8") as f:
        contest_data = json.load(f)
    
    contestants = contest_data.get("contestants", [])
    
    year_dir = os.path.join(contestants_dir, str(year))
    os.makedirs(year_dir, exist_ok=True)

    for contestant in contestants:
        contestant_id = contestant.get("id")
        if not contestant_id:
            continue
        
        url = f"https://eurovisionapi.runasp.net/api/contests/{year}/contestants/{contestant_id}"
        response = requests.get(url)

        if response.status_code == 200:
            contestant_data = response.json()
            file_path = os.path.join(year_dir, f"{contestant_id}.json")
            with open(file_path, "w", encoding="utf-8") as out_file:
                json.dump(contestant_data, out_file, ensure_ascii=False, indent=4)
            print(f"Saved data for contestant {contestant_id} in {year}")
        else:
            print(f"Failed to fetch contestant {contestant_id} for {year}. Status: {response.status_code}")


Saved data for contestant 1 in 1956
Saved data for contestant 2 in 1956
Saved data for contestant 3 in 1956
Saved data for contestant 4 in 1956
Saved data for contestant 5 in 1956
Saved data for contestant 6 in 1956
Saved data for contestant 7 in 1956
Saved data for contestant 8 in 1956
Saved data for contestant 9 in 1956
Saved data for contestant 10 in 1956
Saved data for contestant 11 in 1956
Saved data for contestant 12 in 1956
Saved data for contestant 13 in 1956
Saved data for contestant 1 in 1957
Saved data for contestant 2 in 1957
Saved data for contestant 3 in 1957
Saved data for contestant 4 in 1957
Saved data for contestant 5 in 1957
Saved data for contestant 6 in 1957
Saved data for contestant 7 in 1957
Saved data for contestant 8 in 1957
Saved data for contestant 9 in 1957
Saved data for contestant 1 in 1958
Saved data for contestant 2 in 1958
Saved data for contestant 3 in 1958
Saved data for contestant 4 in 1958
Saved data for contestant 5 in 1958
Saved data for contestan

In [ ]:
import os
import json
import csv

contest_dir = "contests_details"
edges = []

for year in range(2016, 2025):
    contest_path = os.path.join(contest_dir, f"contest_{year}.json")
    if not os.path.exists(contest_path):
        print(f"Contest file for {year} not found.")
        continue

    with open(contest_path, "r", encoding="utf-8") as f:
        contest_data = json.load(f)

    contestant_id_to_country = {
        c["id"]: c["country"] for c in contest_data.get("contestants", [])
    }

    final_round = next(
        (r for r in contest_data.get("rounds", []) if r.get("name", "").lower() == "final"),
        None
    )

    if not final_round:
        print(f"No final round for {year}")
        continue

    performances = final_round.get("performances")
    if performances is None:
        print(f"No performances found for final round in {year}")
        continue

    for perf in performances:
        contestant_id = perf["contestantId"]
        target_country = contestant_id_to_country.get(contestant_id, f"UNKNOWN_{contestant_id}")

        for score in perf.get("scores", []):
            if score.get("name") == "public":
                votes = score.get("votes", {})
                for source_code, vote_count in votes.items():
                    source_country = contestant_id_to_country.get(source_code, source_code)
                    edges.append((source_country, target_country, vote_count))

print(f"Total public vote edges from 2016–2024 finals: {len(edges)}")

# Save edges to CSV
with open("eurovision_public_votes_2016_2024_finals.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["source", "target", "votes"])
    writer.writerows(edges)

print("Edge list saved to eurovision_public_votes_2016_2024_finals.csv")


No performances found for final round in 2020
Total public vote edges from 2016–2024 finals: 8114
Edge list saved to eurovision_public_votes_2016_2024_finals.csv
